In [1]:
import pandas as pd
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.us_income_in_us_db

# Declare the collection
collection = db.us_income_in_us_db

# Extract

## Our original sources came as CSV files from Kaggle. We found sources that had the median incomes for the United States and the other one had locations of Starbucks' in the world.

In [3]:
csv_file = "Resources/starbucks.csv"
starbucks_df = pd.read_csv(csv_file)
starbucks_df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


# Transform

## We took out just a few columns from the Starbuck location data to narrow it down, renamed them, and set the index as the store number. Then we filted out just the results in the United States.

In [4]:
# Create a filtered dataframe from specific columns
starbucks_cols = ["Store Number", "Store Name", "City", "State/Province", "Country"]
starbucks_transformed= starbucks_df[starbucks_cols].copy()

# Rename the column headers
starbucks_transformed = starbucks_transformed.rename(columns={"Store Number": "id",
                                                          "Store Name": "Store_name",
                                                          "State/Province": "State"})

# Clean the data by dropping duplicates and setting the index
starbucks_transformed.drop_duplicates("id", inplace=True)
starbucks_transformed.set_index("id", inplace=True)



In [5]:
columns = [
     "Store_name", "City", "State", "Country"]


starbucks_in_us = starbucks_transformed.loc[starbucks_transformed["Country"] == "US",  columns]
starbucks_in_us = starbucks_in_us.dropna()
starbucks_in_us

,Store_name,City,State,Country
id,,,,
3513-125945,Safeway-Anchorage #1809,Anchorage,AK,US
74352-84449,Safeway-Anchorage #2628,Anchorage,AK,US
12449-152385,Safeway - Anchorage #1813,Anchorage,AK,US
24936-233524,100th & C St - Anchorage,Anchorage,AK,US
8973-85630,Old Seward & Diamond,Anchorage,AK,US
...,...,...,...,...
74385-87621,Safeway-Laramie #2466,Laramie,WY,US
73320-24375,Ridley's - Laramie #1131,Laramie,WY,US
22425-219024,Laramie - Grand & 30th,Laramie,WY,US


## From there we decided to change the index to the State instead.  This allowed us to get the count of the Starbucks stores by state so that we could have an even comparison to the state column from the US income file.

In [6]:
starbucks_in_us.set_index("State", inplace=True)

In [7]:
starCount = starbucks_in_us.groupby("State").count()

starCount = starCount.drop(columns=["Country"])
starCount = starCount.rename(columns={"Store_name": "Starbucks Count"})
starCount


,Starbucks Count,City
State,,
AK,49,49
AL,85,85
AR,55,55
AZ,488,488
CA,2821,2821
CO,481,481
CT,123,123
DC,91,91
DE,25,25


## We imported the US Median income csv and filted it down to a few necessary columns and renamed them to names that were easier to use.  We grouped it by state and then found the Median of the median incomes for each state.

In [8]:
csv_file2 = "Resources/us_income.csv"
us_income_df = pd.read_csv(csv_file2, encoding = "ISO-8859-1")
us_income_df.head()

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328


In [9]:
# Create a filtered dataframe from specific columns
us_income_cols = ["id", "State_ab", "City", "Type", "Median"]
us_income_transformed= us_income_df[us_income_cols].copy()

# Rename the column headers
us_income_transformed = us_income_transformed.rename(columns={"State_ab": "State",
                                                              "Type": "State_type"})

# Clean the data by dropping duplicates and setting the index
us_income_transformed.drop_duplicates("id", inplace=True)
us_income_transformed.set_index("id", inplace=True)

In [10]:
meanMedian = us_income_transformed.groupby(us_income_transformed["State"]).median()
meanMedian

,Median
State,
AK,70536.0
AL,39943.0
AR,37758.0
AZ,47784.0
CA,63367.5
CO,61376.0
CT,79391.0
DC,73188.0
DE,58866.5


## Finally we merged the two DataFrames on the state column, leaving us with a table that shows how many Starbucks are in each state and the Median median income of that state.

In [11]:
Combined = pd.merge(starCount, meanMedian, how ="outer", on ="State")
Combined = Combined.dropna()
Combined = Combined.drop(columns="City")
Combined["Starbucks Count"] = Combined["Starbucks Count"].map('{:.0f}'.format)
Combined["Median"] = Combined["Median"].map('${:,.2f}'.format)
Combined


,Starbucks Count,Median
State,,
AK,49,"$70,536.00"
AL,85,"$39,943.00"
AR,55,"$37,758.00"
AZ,488,"$47,784.00"
CA,2821,"$63,367.50"
CO,481,"$61,376.00"
CT,123,"$79,391.00"
DC,91,"$73,188.00"
DE,25,"$58,866.50"


# Load

## We chose to load it into a mongo database since it's easier to use becuase you don't have to create the tables ahead of time, they can just be created when the information is loaded into them.

In [12]:
# Insert the document into the database
# The database and collection, if they don't already exist, will be created at this point.
db.collection.insert_many(Combined.to_dict('records'))